# Preprocessing Arizona Allocation data for WaDEQA upload.
Purpose:  To preprocess the Arizona data into one mail file for simple DataFrame creation and extraction.

In [ ]:
#Needed Libararies

# working with data
import os
import numpy as np
import pandas as pd
import geopandas as gpd

# visulizaiton
import matplotlib.pyplot as plt
import seaborn as sns

# API retrieval
import requests
import json

# Cleanup
import re
from datetime import datetime
pd.set_option('display.max_columns', 999)  # How to display all columns of a Pandas DataFrame in Jupyter Notebook
pd.set_option('display.float_format', lambda x: '%.5f' % x) # suppress scientific notation in Pandas

In [ ]:
# Working Directory
workingDir = "G:/Shared drives/WaDE Data/Arizona/WaterAllocation/RawInputData"
os.chdir(workingDir)

## Groundwater Data (POD)

In [ ]:
# input File - Well_Registry.csv
fileInput = "Groundwater/Well_Registry.zip"
dfgw = pd.read_csv(fileInput, compression='zip').replace(np.nan, "")

if 'WaDEUUID' not in dfgw:
    dfgw['WaDEUUID'] = "azGW" + dfgw.index.astype(str)
    dfgw.to_csv('Groundwater/Well_Registry.zip', compression='zip', index=False)

print(len(dfgw))
dfgw.head()

In [ ]:
# AZwr Groundwater PUMPRATE is in GPM, need to convert to CFS
# 448.8 CFS = 1 GPM

# Clean owner name up
def ConvertGPMToCFSFunc(Val):
    Val = Val / 448.8 
    return Val

dfgw['PUMPRATE'] = dfgw.apply(lambda row: ConvertGPMToCFSFunc(row['PUMPRATE']), axis=1)
dfgw['PUMPRATE'].unique()

In [ ]:
# create output POD dataframe
df = pd.DataFrame()

# Data Assessment UUID
df['WaDEUUID'] = dfgw['WaDEUUID']

# WaterSource Info
df['in_Geometry'] = ""
df['in_GNISFeatureNameCV'] = ""
df['in_WaterQualityIndicatorCV'] = ""
df['in_WaterSourceName'] = ""
df['in_WaterSourceNativeID'] = "" # create customID for temp solution
df['in_WaterSourceTypeCV'] = "Groundwater"

# Site Info
df['in_RegulatoryOverlayUUIDs'] = ""
df['in_WaterSourceUUID'] = "" # ???
df['in_CoordinateAccuracy'] = ""
df['in_CoordinateMethodCV'] = ""
df['in_County'] = dfgw['COUNTY'].str.title()
df['in_EPSGCodeCV'] = 4326
df['in_Geometry'] = ""
df['in_GNISCodeCV'] = ""
df['in_HUC12'] = ""
df['in_HUC8'] = ""
df['in_Latitude'] = dfgw['latitude']
df['in_Longitude'] = dfgw['longitude']
df['in_NHDNetworkStatusCV'] = ""
df['in_NHDProductCV'] = ""
df['in_PODorPOUSite'] = "POD"
df['in_SiteName'] = ""
df['in_SiteNativeID'] = "POD" + dfgw['CADASTRAL'].replace("", 0).fillna(0).astype(str)
df['in_SitePoint'] = ""
df['in_SiteTypeCV'] = "Well" # these should all be well records
df['in_StateCV'] = "AZ"
df['in_USGSSiteID'] = ""

# AllocationAmount Info
df['in_MethodUUID'] = "AZwr_M1" # for goundwater
df['in_OrganizationUUID'] = "AZwr_O1"
df['in_SiteUUID'] = "" # ???
df['in_VariableSpecificUUID'] =  "AZwr_V1" # for CFS
df['in_AllocationApplicationDate'] = ""
df['in_AllocationAssociatedConsumptiveUseSiteIDs'] = ""
df['in_AllocationAssociatedWithdrawalSiteIDs'] = ""
df['in_AllocationBasisCV'] = ""
df['in_AllocationChangeApplicationIndicator'] = ""
df['in_AllocationCommunityWaterSupplySystem'] = ""
df['in_AllocationCropDutyAmount'] = ""
df['in_AllocationExpirationDate'] = ""
df['in_AllocationFlow_CFS'] = dfgw['PUMPRATE'].astype(float)
df['in_AllocationLegalStatusCV'] = ""
df['in_AllocationNativeID'] =  dfgw['REGISTRY_I'].replace("", 0).fillna(0).astype(int).astype(str)
df['in_AllocationOwner'] = dfgw['OWNER_NAME']
df['in_AllocationPriorityDate'] = ""
df['in_AllocationSDWISIdentifierCV'] = ""
df['in_AllocationTimeframeEnd'] = "12/31"
df['in_AllocationTimeframeStart'] = "01/01"
#df['in_AllocationTypeCV'] = dfgw['WELL_TYPE_'] # skip for now
df['in_AllocationTypeCV'] = ""
df['in_AllocationVolume_AF'] = ""
df['in_BeneficialUseCategory'] = dfgw['WATER_USE'].str.title()
df['in_CommunityWaterSupplySystem'] = ""
df['in_CropTypeCV'] = ""
df['in_CustomerTypeCV'] = ""
df['in_DataPublicationDate'] = ""
df['in_DataPublicationDOI'] = ""
df['in_ExemptOfVolumeFlowPriority'] = 1 # all these gw records should be considered exempt for us.
df['in_GeneratedPowerCapacityMW'] = ""
df['in_IrrigatedAcreage'] = ""
df['in_IrrigationMethodCV'] = ""
df['in_LegacyAllocationIDs'] = ""
df['in_OwnerClassificationCV'] = ""
df['in_PopulationServed'] = ""
df['in_PowerType'] = ""
df['in_PrimaryBeneficialUseCategory'] = ""
df['in_SDWISIdentifierCV'] = ""
df['in_WaterAllocationNativeURL'] = "https://app.azwater.gov/WellRegistry/Detail.aspx?" + dfgw['REGISTRY_I'].replace("", 0).fillna(0).astype(int).astype(str)

dfgwOut = df.copy()
print(len(dfgwOut))
dfgwOut.head()

## Surface Water Data (POD & POU) 

In [ ]:
#Input files - Surface Water Query by Watershed water records

# Surface Water Query by Watershed water record inputs.
csv_file_list = [
    "Surface Water/SW QUERY BY SURFACE WATERSHEDS/AGUA FRIA RIVER.zip",
    "Surface Water/SW QUERY BY SURFACE WATERSHEDS/BILL WILLIAMS RIVER.zip",
    "Surface Water/SW QUERY BY SURFACE WATERSHEDS/COLORADO RIVER.zip",
    "Surface Water/SW QUERY BY SURFACE WATERSHEDS/LITTLE COLORADO RIVER.zip",
    "Surface Water/SW QUERY BY SURFACE WATERSHEDS/LOWER GILA RIVER.zip",
    "Surface Water/SW QUERY BY SURFACE WATERSHEDS/RIO YAQUI.zip",
    "Surface Water/SW QUERY BY SURFACE WATERSHEDS/SALT RIVER.zip",
    "Surface Water/SW QUERY BY SURFACE WATERSHEDS/SAN PEDRO RIVER.zip",
    "Surface Water/SW QUERY BY SURFACE WATERSHEDS/SAN SIMON RIVER.zip",
    "Surface Water/SW QUERY BY SURFACE WATERSHEDS/SANTA CRUZ RIVER.zip",
    "Surface Water/SW QUERY BY SURFACE WATERSHEDS/UPPER GILA RIVER.zip",
    "Surface Water/SW QUERY BY SURFACE WATERSHEDS/VERDE RIVER.zip",
    "Surface Water/SW QUERY BY SURFACE WATERSHEDS/VIRGIN RIVER.zip",
    "Surface Water/SW QUERY BY SURFACE WATERSHEDS/WHITE WATER DRAW.zip",
    "Surface Water/SW QUERY BY SURFACE WATERSHEDS/WILLCOX PLAYA.zip"]


list_of_dataframes = []
for filename in csv_file_list:
    list_of_dataframes.append(pd.read_csv(filename, compression='zip'))

dfsw = pd.concat(list_of_dataframes).replace(np.nan, "").drop_duplicates().reset_index(drop=True)
dfsw = dfsw.drop(['LEGAL'], axis=1).drop_duplicates().reset_index(drop=True) # drop 'LEGAL', not needed.

if 'WaDEUUID' not in dfsw:
    dfsw['WaDEUUID'] = "azSW" + dfsw.index.astype(str)
    dfsw.to_csv('Surface Water/SW_QUERY_COMBINED.zip', compression=dict(method='zip', archive_name='SW_QUERY_COMBINED.csv'), index=False) 

    
dfsw = dfsw.drop_duplicates().reset_index(drop=True).replace(np.nan, "")
print(len(dfsw))
dfsw.head(3)

In [ ]:
# fixing 'REG. NO' format to match 'FILNO' in FILINGS shp file.

def fixREGNO(val):
   
    ### first fix
    # Create testVal to search for length, split on '-' & '.'
    testVal = str(val).strip()
    sep1 = '-'
    testVal = testVal.split(sep1, 1)[1]
    sep2 = '.'
    testVal = testVal.split(sep2, 1)[0]
    
    # inerst new text into 'val' based on 'testVal' length.
    if len(testVal) == 2:
        val = val.replace("-", "-0000")
    if len(testVal) == 3:
        val = val.replace("-", "-000")
    if len(testVal) == 4:
        val = val.replace("-", "-00")
    if len(testVal) == 5:
        val = val.replace("-", "-0")
        
        
    ### second fix
    # Create testVal to search for length, split on '.' at the end
    testVal = str(val).strip()
    sep1 = '.'
    testVal = testVal.split(sep1, 1)[1]
    
    # inerst new text into 'val' based on 'testVal' length.
    if len(testVal) == 1:
        val = val.replace(".", ".00" + testVal)
    if len(testVal) == 2:
        val = val.replace(".", ".0" + testVal)   
    
    return val

dfsw['REG. NO'] = dfsw.apply(lambda row: fixREGNO(row['REG. NO']), axis=1)
exList = dfsw['REG. NO'].unique().tolist()
exList.sort()
for x in exList:
    print(x)

In [ ]:
#Input files - Fillings Layers.shp

#POD layer
fileName = "Surface Water/shapefile/FilingPOD.zip"
df_PODfill = pd.read_csv(fileName, compression='zip')

#POU layer
fileName = "Surface Water/shapefile/FillingPOU.zip"
df_POUfill = pd.read_csv(fileName, compression='zip')

# Concatenate dataframes
frames = [df_PODfill, df_POUfill] # add dataframes here
df_fill = pd.concat(frames)
df_fill = df_fill.drop_duplicates().reset_index(drop=True).replace(np.nan, "")
print(len(df_fill))
df_fill.head(1)

In [ ]:
# Merge Surface Water Query by Watershed water records with Filling.
dfsw = pd.merge(dfsw, df_fill, left_on='REG. NO', right_on='FILENO', how='left')
print(len(dfsw))
dfsw.head(1)

In [ ]:
# Split 'QUANTITY' into 'Amount' and 'UNIT'
dfsw[['Amount', 'Unit']] = dfsw.QUANTITY.str.split("  ", expand = True)
dfsw['Amount'] = pd.to_numeric(dfsw['Amount'], errors='coerce').fillna(0).astype(float) # make sure this is numeric.
dfsw.head(1)

In [ ]:
# 03/02/2023
# temp fix - remove recods with these 'Units'
dropList = ['ACRES',
            'Amount Required for Maintenance',
            'Feet',
            'MIT - Miners Inches Total',
            'Miners Inches Per Annum', 
            'XX - Unknown Code at Load time',
            'None',
            '',
            " "]

dfsw = dfsw[~dfsw['Unit'].isin(dropList)]
print(len(dfsw))
dfsw.head(1)

In [ ]:
# Need to update to reflect groundwater values too
# CFS = V1, AF = V2

# Create VariableSpecificCv value
def createVariableSpecificCv(unit):
    outString = ""
    if unit == "Cubic Feet Per Second":
        outString = "AZwr_V1"
    if unit == "Acre-Feet Per Annum":
        outString = "AZwr_V1"
    if unit == "Gallons Per Annum":
        outString = "AZwr_V1"
    else:
        outString = "AZwr_V2"

    return(outString)

dfsw['in_VariableSpecificUUID'] = dfsw.apply(lambda row: createVariableSpecificCv(row['Unit']), axis=1)
dfsw['in_VariableSpecificUUID'].unique()

In [ ]:
# convert all flow values to CFS
def convertFlowFunc(val, unit):
    CFS_Value = None
    if unit == "Cubic Feet Per Second":
        CFS_Value = val
    if unit == "Acre-Feet Per Annum":
        CFS_Value = val / (723.968)
    if unit == "Gallons Per Annum":
        CFS_Value = val / (235905662.34)
    else:
        CFS_Value = 0.0
    return(CFS_Value)

dfsw['CFS_Value'] = dfsw.apply(lambda row: convertFlowFunc(row['Amount'], row['Unit']), axis=1)
dfsw['CFS_Value'].unique()

In [ ]:
# convert all volume values to AF
def convertVolumeFunc(val, unit):
    AF_Value = None
    if unit == 'Acre-Feet':
        AF_Value = val
    if unit == 'Acre-Feet Total':
        AF_Value = val
    if unit == "CFT - Cubic Feet Total":
        AF_Value = val / (43559.9)
    if unit == 'Gallons':
        AF_Value = val / (325850.943)
    else:
        AF_Value = 0.0
    return(AF_Value)

dfsw['AF_Value'] = dfsw.apply(lambda row: convertVolumeFunc(row['Amount'], row['Unit']), axis=1)
dfsw['AF_Value'].unique()

In [ ]:
# Creating long and lat values from data.  
# Need to convert from UTM 12N to WGS 84.
# I believe AZ is consiered WGS 84 / UTM zone 12N - EPSG:32612.

from pyproj import Proj
myProj = Proj(proj='utm',zone=12, ellps='WGS84', preserve_units=False)
long, lat = myProj(dfsw['X_UTMNAD83'].values, dfsw['Y_UTMNAD83'].values, inverse=True)
dfsw['in_Latitude'] = lat
dfsw['in_Longitude'] = long
dfsw = dfsw.replace(np.nan, '')  # Replaces NaN values with blank.
dfsw.head(1)

In [ ]:
# create output POD dataframe
df = pd.DataFrame()

# Data Assessment UUID
df['WaDEUUID'] = dfsw['WaDEUUID']

# WaterSource Info
df['in_Geometry'] = ""
df['in_GNISFeatureNameCV'] = ""
df['in_WaterQualityIndicatorCV'] = ""
df['in_WaterSourceName'] = dfsw['WATERSOURC'].str.title()
df['in_WaterSourceNativeID'] = "" # create customID for temp solution
df['in_WaterSourceTypeCV'] = "Surface Water"

# Site Info
df['in_RegulatoryOverlayUUIDs'] = ""
df['in_WaterSourceUUID'] = "" # ???
df['in_CoordinateAccuracy'] = ""
df['in_CoordinateMethodCV'] = ""
df['in_County'] = dfsw['COUNTY'].str.title()
df['in_EPSGCodeCV'] = 4326
df['in_Geometry'] = ""
df['in_GNISCodeCV'] = ""
df['in_HUC12'] = ""
df['in_HUC8'] = ""
df['in_Latitude'] = dfsw['in_Latitude']
df['in_Longitude'] = dfsw['in_Longitude']
df['in_NHDNetworkStatusCV'] = ""
df['in_NHDProductCV'] = ""
df['in_PODorPOUSite'] = dfsw['POU_POD']
df['in_SiteName'] = ""
df['in_SiteNativeID'] = dfsw['POU_POD'].str.strip() + dfsw['CADASTRAL'].replace("", 0).fillna(0).astype(str)
df['in_SitePoint'] = ""
df['in_SiteTypeCV'] = ""
df['in_StateCV'] = "AZ"
df['in_USGSSiteID'] = ""

# AllocationAmount Info
df['in_MethodUUID'] = "AZwr_M2" # for surface water
df['in_OrganizationUUID'] = "AZwr_O1"
df['in_SiteUUID'] = "" # ???
df['in_VariableSpecificUUID'] =  dfsw['in_VariableSpecificUUID']
df['in_AllocationApplicationDate'] = ""
df['in_AllocationAssociatedConsumptiveUseSiteIDs'] = ""
df['in_AllocationAssociatedWithdrawalSiteIDs'] = ""
df['in_AllocationBasisCV'] = ""
df['in_AllocationChangeApplicationIndicator'] = ""
df['in_AllocationCommunityWaterSupplySystem'] = ""
df['in_AllocationCropDutyAmount'] = ""
df['in_AllocationExpirationDate'] = ""
df['in_AllocationFlow_CFS'] = dfsw['CFS_Value'].astype(float) # see above for conversion
df['in_AllocationLegalStatusCV'] = dfsw['STATUS_x'].str.title()
df['in_AllocationNativeID'] =  dfsw['REG. NO'].replace("", 0).fillna(0).astype(str)
df['in_AllocationOwner'] = dfsw['NAME']
df['in_AllocationPriorityDate'] = dfsw['PRIOR DATE']
df['in_AllocationSDWISIdentifierCV'] = ""
df['in_AllocationTimeframeEnd'] = "12/31"
df['in_AllocationTimeframeStart'] = "01/01"
df['in_AllocationTypeCV'] = ""
df['in_AllocationVolume_AF'] = dfsw['AF_Value'].astype(float) # see above for conversion
df['in_BeneficialUseCategory'] = dfsw['WATER USE'].str.title()
df['in_CommunityWaterSupplySystem'] = ""
df['in_CropTypeCV'] = ""
df['in_CustomerTypeCV'] = ""
df['in_DataPublicationDate'] = ""
df['in_DataPublicationDOI'] = ""
df['in_ExemptOfVolumeFlowPriority'] = 0
df['in_GeneratedPowerCapacityMW'] = ""
df['in_IrrigatedAcreage'] = ""
df['in_IrrigationMethodCV'] = ""
df['in_LegacyAllocationIDs'] = ""
df['in_OwnerClassificationCV'] = ""
df['in_PopulationServed'] = ""
df['in_PowerType'] = ""
df['in_PrimaryBeneficialUseCategory'] = ""
df['in_SDWISIdentifierCV'] = ""
df['in_WaterAllocationNativeURL'] = ""

dfswOut = df.copy()
print(len(dfswOut))
dfswOut.head()

## Concatenate GW with SW dataframes

In [ ]:
frames = [dfgwOut, dfswOut] # add dataframes here
outdf = pd.concat(frames)
outdf = outdf.drop_duplicates().reset_index(drop=True).replace(np.nan, "")
print(len(outdf))

## Clean output dataframes

In [ ]:
# Clean owner name up
def cleanOwnerDataFunc(Val):
    Val = str(Val)
    Val = re.sub("[$@&.;,/\)(-]", "", Val).title().strip()
    return Val

In [ ]:
outdf['in_AllocationOwner'] = outdf.apply(lambda row: cleanOwnerDataFunc(row['in_AllocationOwner']), axis=1)
outdf['in_AllocationOwner'].unique()

In [ ]:
outdf['in_WaterSourceName'] = outdf.apply(lambda row: cleanOwnerDataFunc(row['in_WaterSourceName']), axis=1)
outdf['in_WaterSourceName'].unique()

In [ ]:
outdf['in_SiteName'] = outdf.apply(lambda row: cleanOwnerDataFunc(row['in_SiteName']), axis=1)
outdf['in_SiteName'].unique()

In [ ]:
outdf['in_County'] = outdf.apply(lambda row: cleanOwnerDataFunc(row['in_County']), axis=1)
outdf['in_County'].unique()

In [ ]:
# Ensure Empty String

def ensureEmptyString(val):
    val = str(val).strip()
    if val == "" or val == " " or val == "nan" or pd.isnull(val):
        outString = ""
    else:
        outString = val
    return outString

In [ ]:
outdf['in_WaterSourceName'] = outdf.apply(lambda row: ensureEmptyString(row['in_WaterSourceName']), axis=1)
outdf['in_WaterSourceName'].unique()

In [ ]:
outdf['in_WaterSourceTypeCV'] = outdf.apply(lambda row: ensureEmptyString(row['in_WaterSourceTypeCV']), axis=1)
outdf['in_WaterSourceTypeCV'].unique()

In [ ]:
outdf['in_County'] = outdf.apply(lambda row: ensureEmptyString(row['in_County']), axis=1)
outdf['in_County'].unique()

In [ ]:
outdf['in_SiteName'] = outdf.apply(lambda row: ensureEmptyString(row['in_SiteName']), axis=1)
outdf['in_SiteName'].unique()

In [ ]:
outdf['in_SiteTypeCV'] = outdf.apply(lambda row: ensureEmptyString(row['in_SiteTypeCV']), axis=1)
outdf['in_SiteTypeCV'].unique()

In [ ]:
outdf['in_AllocationLegalStatusCV'] = outdf.apply(lambda row: ensureEmptyString(row['in_AllocationLegalStatusCV']), axis=1)
outdf['in_AllocationLegalStatusCV'].unique()

In [ ]:
outdf['in_AllocationOwner'] = outdf.apply(lambda row: ensureEmptyString(row['in_AllocationOwner']), axis=1)
outdf['in_AllocationOwner'].unique()

In [ ]:
outdf['in_BeneficialUseCategory'] = outdf.apply(lambda row: ensureEmptyString(row['in_BeneficialUseCategory']), axis=1)
outdf['in_BeneficialUseCategory'].unique()

In [ ]:
# in_Latitude
outdf['in_Latitude'] = pd.to_numeric(outdf['in_Latitude'], errors='coerce').fillna("")
outdf['in_Latitude'].unique()

In [ ]:
# in_Longitude
outdf['in_Longitude'] = pd.to_numeric(outdf['in_Longitude'], errors='coerce').fillna("")
outdf['in_Longitude'].unique()

In [ ]:
#Update datatype of Priority Date to fit WaDE 2.0 structure
outdf['in_AllocationPriorityDate'] = pd.to_datetime(outdf['in_AllocationPriorityDate'], errors = 'coerce')
outdf['in_AllocationPriorityDate'] = pd.to_datetime(outdf['in_AllocationPriorityDate'].dt.strftime('%m/%d/%Y'))
outdf['in_AllocationPriorityDate'].unique()

In [ ]:
# Fixing in_AllocationFlow_CFS datatype
outdf['in_AllocationFlow_CFS'] = pd.to_numeric(outdf['in_AllocationFlow_CFS'], errors='coerce').replace(0,"").fillna("")
outdf['in_AllocationFlow_CFS'].unique()

In [ ]:
# Fixing in_AllocationVolume_AF datatype
outdf['in_AllocationVolume_AF'] = pd.to_numeric(outdf['in_AllocationVolume_AF'], errors='coerce').replace(0,"").fillna("")
outdf['in_AllocationVolume_AF'].unique()

In [ ]:
# Creating WaDE Custom water source native ID for easy water source identification
# ----------------------------------------------------------------------------------------------------

# Create temp WaterSourceNativeID dataframe of unique water source.
def assignWaterSourceNativeID(colrowValue):
    string1 = str(colrowValue)
    outstring = "wadeID" + string1
    return outstring

dfWaterSourceNativeID = pd.DataFrame()
dfWaterSourceNativeID['in_WaterSourceName'] = outdf['in_WaterSourceName']
dfWaterSourceNativeID['in_WaterSourceTypeCV'] = outdf['in_WaterSourceTypeCV']
dfWaterSourceNativeID = dfWaterSourceNativeID.drop_duplicates()

dftemp = pd.DataFrame(index=dfWaterSourceNativeID.index)
dftemp["Count"] = range(1, len(dftemp.index) + 1)
dfWaterSourceNativeID['in_WaterSourceNativeID'] = dftemp.apply(lambda row: assignWaterSourceNativeID(row['Count']), axis=1)
dfWaterSourceNativeID['linkKey'] = dfWaterSourceNativeID['in_WaterSourceName'].astype(str) + dfWaterSourceNativeID['in_WaterSourceTypeCV'].astype(str)

# ----------------------------------------------------------------------------------------------------

# Retreive WaDE Custom water source native ID
WaterSourceNativeIDdict = pd.Series(dfWaterSourceNativeID.in_WaterSourceNativeID.values, index=dfWaterSourceNativeID.linkKey.astype(str)).to_dict()
def retrieveWaterSourceNativeID(A, B):
    if (A == '' and B == '') or (pd.isnull(A) and pd.isnull(B)):
        outList = ''
    else:
        colrowValue = str(A).strip() + str(B).strip()
        try:
            outList = WaterSourceNativeIDdict[colrowValue]
        except:
            outList = ''
    return outList

outdf['in_WaterSourceNativeID'] = outdf.apply(lambda row: retrieveWaterSourceNativeID( row['in_WaterSourceName'], row['in_WaterSourceTypeCV']), axis=1)
outdf['in_WaterSourceNativeID'].unique()

## Drop non-Active AllocationLegalStatusCV Water Rights
- For this ADWR, we don't want water rights that are considered: "INACTIVE - WITHDRAWN",
                       "INACTIVE - CONSOLIDATED",
                       "INACTIVE - AMENDED",
                       "INACTIVE - CANCELLED",
                       "INACTIVE - REJECTED",
                       "INACTIVE - PARTIAL T&S",
                       "INACTIVE - RELINQUISHED",
                       "INACTIVE - FULL T&S",
                       "INACTIVE - INACTIVE",
                       "INACTIVE - FULL ASSIGNMENT",
                       "INACTIVE - PARTIAL ASSIGNMENT"

In [ ]:
# drop non-active AllocationLegalStatusCV values specific to that state.

# drop the list
dropLegalStatusList = ["INACTIVE - WITHDRAWN",
                       "INACTIVE - CONSOLIDATED",
                       "INACTIVE - AMENDED",
                       "INACTIVE - CANCELLED",
                       "INACTIVE - REJECTED",
                       "INACTIVE - PARTIAL T&S",
                       "INACTIVE - RELINQUISHED",
                       "INACTIVE - FULL T&S",
                       "INACTIVE - INACTIVE",
                       "INACTIVE - FULL ASSIGNMENT",
                       "INACTIVE - PARTIAL ASSIGNMENT"] # enter string entries here

# drop rows from above list
outdf = outdf[outdf.in_AllocationLegalStatusCV.isin(dropLegalStatusList) == False].reset_index(drop=True)

print(len(outdf))
outdf['in_AllocationLegalStatusCV'].unique()

## Shapefile Data
- For attaching geometry to POU csv inputs.

In [ ]:
# no shp file data to work with AZ wr.

In [ ]:
# # PoU Shapefile Data
# shapefileInput = "RawInputData/shapefiles/{enter file name here}.zip" # ziped folder of the shp file
# dfPoUshapetemp = gpd.read_file(shapefileInput)
# print(len(dfPoUshapetemp))
# dfPoUshapetemp.head()

In [ ]:
# # create temp dataframe to hold native ID and geometry from shapefile input
# columnsList = ['in_SiteNativeID', 'geometry']
# dfPoUshape = pd.DataFrame(columns=columnsList)

# # assing values to temp dataframe based on shapefile input
# # for in_SiteNativeID assure ID value is the same as that listed above for POU info.
# dfPoUshape['in_SiteNativeID'] = "POU" + ""
# dfPoUshape['geometry'] = dfPoUshapetemp['geometry']
# dfPoUshape = dfPoUshape.drop_duplicates(subset=None, keep='first', inplace=False, ignore_index=False)
# print(len(dfPoUshape))
# dfPoUshape.head()

## Review and Export

In [ ]:
outdf.info()

In [ ]:
outdf

In [ ]:
# Export the output dataframe
outdf.to_csv('Pwr_azMain.zip', compression=dict(method='zip', archive_name='Pwr_azMain.csv'), index=False) # The output, save as a zip
#dfPoUshape.to_csv('P_Geometry.zip', compression=dict(method='zip', archive_name='P_Geometry.csv'), index=False)  # The output geometry.